<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [5]:
# load in the appropriate path to the file you want to load in
rockfile = "../../data/rock.csv"

In [12]:
df = pd.read_csv(rockfile)

In [13]:
df.columns.tolist()

['Song Clean',
 'ARTIST CLEAN',
 'Release Year',
 'COMBINED',
 'First?',
 'Year?',
 'PlayCount',
 'F*G']

### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [11]:
df = pd.read_csv(rockfile, names = ['song','artist','release','combined','first','year','plays','f*g'])
df.head()

,song,artist,year released,combined,first,year,plays,f*g
0,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
1,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
2,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
3,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
4,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [60]:
df.rename(columns = {
    'Song Clean': 'song',
    'ARTIST CLEAN': 'artist',
    'Release Year': 'release',
    'COMBINED': 'combined',
    'First?': 'first',
    'Year?' : 'year',
    'PlayCount': 'plays',
    'F*G': 'f*g'
}).head()

,song,artist,release,combined,first,year,plays,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [18]:
df.columns = ['song','artist','release','combined','first','year','plays','f*g']

### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [20]:
df[df['release'].isnull()].head()

,song,artist,release,combined,first,year,plays,f*g
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0


#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [21]:
df.loc[df['release'].isnull(),'release'] = 0

In [61]:
df.select_dtypes(include = np.number).columns

Index(['release', 'first', 'year', 'plays', 'f*g'], dtype='object')

#### 4.B Verify that `release` contains no null values.

In [23]:
df['release'].isnull().sum()

0

### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
song        2230 non-null object
artist      2230 non-null object
release     2230 non-null object
combined    2230 non-null object
first       2230 non-null int64
year        2230 non-null int64
plays       2230 non-null int64
f*g         2230 non-null int64
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [25]:
df['release'].unique()

array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1071', '1996', '1991', '2005', '2011', '2004', '2012', '2003',
       '1998', '2008', '1964', '2013', '2006', 'SONGFACTS.COM', '1963',
       '1961'], dtype=object)

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [26]:
df[df['release'] == 'SONGFACTS.COM']

,song,artist,release,combined,first,year,plays,f*g
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [64]:
#df.loc[df['release'] == 'SONGFACTS.COM', 'release'] = 0
df['release'] = df['release'].astype(np.int16)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
song        2230 non-null object
artist      2230 non-null object
release     2230 non-null int16
combined    2230 non-null object
first       2230 non-null int64
year        2230 non-null int64
plays       2230 non-null int64
f*g         2230 non-null int64
dtypes: int16(1), int64(4), object(3)
memory usage: 126.4+ KB


### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [65]:
df[df['release'] != 0]['release'].describe()

count    1652.000000
mean     1978.564770
std         9.308823
min      1955.000000
25%      1971.000000
50%      1977.000000
75%      1984.000000
max      2014.000000
Name: release, dtype: float64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [38]:
df[df['release'] == 1071]

,song,artist,release,combined,first,year,plays,f*g
547,Levon,Elton John,1071,Levon by Elton John,1,1,8,8


_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

In [40]:
df.loc[df['release'] == 1071, 'release'] = 1971

### 8. Make changes and investigate using custom functions with `.apply()`.


Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [57]:
def track_facts(track):
    song = track['song']
    artist = track['artist']
    before_70 = track['release'] < 1970
    return(song, artist, before_70)

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [66]:
df.head().apply(track_facts, axis = 1)

0          (Caught Up in You, .38 Special, False)
1               (Fantasy Girl, .38 Special, True)
2           (Hold On Loosely, .38 Special, False)
3    (Rockin' Into the Night, .38 Special, False)
4                (Art For Arts Sake, 10cc, False)
dtype: object

In [67]:
df.head()

,song,artist,release,combined,first,year,plays,f*g
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
